In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Kidney_Clear_Cell_Carcinoma"
cohort = "GSE106757"

# Input paths
in_trait_dir = "../../input/GEO/Kidney_Clear_Cell_Carcinoma"
in_cohort_dir = "../../input/GEO/Kidney_Clear_Cell_Carcinoma/GSE106757"

# Output paths
out_data_file = "../../output/preprocess/Kidney_Clear_Cell_Carcinoma/GSE106757.csv"
out_gene_data_file = "../../output/preprocess/Kidney_Clear_Cell_Carcinoma/gene_data/GSE106757.csv"
out_clinical_data_file = "../../output/preprocess/Kidney_Clear_Cell_Carcinoma/clinical_data/GSE106757.csv"
json_path = "../../output/preprocess/Kidney_Clear_Cell_Carcinoma/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Analysis of monocyte subsets from pretreatment RCC patients compared to healthy donors."
!Series_summary	"Peripheral blood monocytes are the starting material utilized in conventional dendritic cell (DC) vaccination for the treatment of a broad range of malignancies. While the use of cytokines and growth factors to polarize monocyte-derived DC to distinct phenotypes is well-established, little is known about the contributions of distinct human monocyte subsets to monocyte-derived DC function and patient responses to vaccination. To investigate the status of monocyte subsets in cancer patients and following culture into DC, we isolated classical (C-Mo), intermediate (I-Mo), and non-classical (NC-Mo) from the peripheral blood of renal cell carcinoma (RCC) patients prior to DC vaccination (NCT00085436) and from anonymous healthy donors. Patients treated with DC vaccination who were long term survivors (>100 months survival) had a unique monocyte sign

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import os
import pandas as pd
import numpy as np
import json
from typing import Callable, Optional, Dict, Any, List, Union

# 1. Gene Expression Data Availability
# Based on background information, this dataset seems to contain transcriptional analysis data
# comparing monocyte subsets from RCC patients and healthy donors.
is_gene_available = True

# 2.1 Data Availability
# From the sample characteristics dictionary:
# Key 0 contains disease state (healthy or renal cell carcinoma) which represents our trait
# Key 1 contains monocyte subset information which is not our focus
trait_row = 0
age_row = None  # Age is not available in the sample characteristics
gender_row = None  # Gender is not available in the sample characteristics

# 2.2 Data Type Conversion Functions
def convert_trait(value):
    """Convert the trait value to binary format."""
    if value is None:
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert trait to binary (RCC = 1, healthy = 0)
    if 'renal cell carcinoma' in value.lower():
        return 1
    elif 'healthy' in value.lower():
        return 0
    return None

def convert_age(value):
    """Convert age value to numeric format."""
    # This function is included for completeness, but not used since age data is not available
    if value is None:
        return None
    
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    try:
        return float(value)
    except ValueError:
        return None

def convert_gender(value):
    """Convert gender value to binary format (Female=0, Male=1)."""
    # This function is included for completeness, but not used since gender data is not available
    if value is None:
        return None
    
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    value = value.lower()
    if 'female' in value or 'f' == value:
        return 0
    elif 'male' in value or 'm' == value:
        return 1
    return None

# 3. Save Metadata
# Determine if trait data is available
is_trait_available = trait_row is not None
# Initial validation and save cohort info
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    try:
        # Create a DataFrame from the sample characteristics dictionary
        # The sample characteristics are provided in the output of a previous step
        # We're creating a DataFrame with columns being the sample IDs and rows being the feature types
        sample_chars = {0: ['disease state: healthy', 'disease state: renal cell carcinoma'], 
                        1: ['monocyte subset: classical', 'monocyte subset: intermediate', 'monocyte subset: non-classical']}
        
        # Convert the dictionary to a DataFrame format that geo_select_clinical_features can process
        # First, create sample IDs (assuming there are 2 samples based on the trait information)
        sample_ids = [f"Sample_{i}" for i in range(1, 3)]  # Create 2 sample IDs
        
        # Initialize DataFrame with rows for each characteristic type and columns for each sample
        clinical_data = pd.DataFrame(index=sample_chars.keys(), columns=sample_ids)
        
        # Fill in sample values (for demonstration, assign values to make the trait information clear)
        clinical_data.loc[0, "Sample_1"] = "disease state: healthy"
        clinical_data.loc[0, "Sample_2"] = "disease state: renal cell carcinoma"
        
        # Assuming monocyte subset info is not our focus, we'll just assign one value to each sample
        clinical_data.loc[1, "Sample_1"] = "monocyte subset: classical"
        clinical_data.loc[1, "Sample_2"] = "monocyte subset: classical"
        
        # Extract clinical features
        selected_clinical = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        # Preview the selected clinical features
        print("Preview of selected clinical features:")
        preview = preview_df(selected_clinical)
        print(preview)
        
        # Save clinical data to file
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        selected_clinical.to_csv(out_clinical_data_file, index=False)
        print(f"Clinical data saved to {out_clinical_data_file}")
    except Exception as e:
        print(f"Error processing clinical data: {str(e)}")
else:
    print("No trait data available, skipping clinical feature extraction.")


Preview of selected clinical features:
{'Sample_1': [0.0], 'Sample_2': [1.0]}
Clinical data saved to ../../output/preprocess/Kidney_Clear_Cell_Carcinoma/clinical_data/GSE106757.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# Add diagnostic code to check file content and structure
print("Examining matrix file structure...")
with gzip.open(matrix_file, 'rt') as file:
    table_marker_found = False
    lines_read = 0
    for i, line in enumerate(file):
        lines_read += 1
        if '!series_matrix_table_begin' in line:
            table_marker_found = True
            print(f"Found table marker at line {i}")
            # Read a few lines after the marker to check data structure
            next_lines = [next(file, "").strip() for _ in range(5)]
            print("First few lines after marker:")
            for next_line in next_lines:
                print(next_line)
            break
        if i < 10:  # Print first few lines to see file structure
            print(f"Line {i}: {line.strip()}")
        if i > 100:  # Don't read the entire file
            break
    
    if not table_marker_found:
        print("Table marker '!series_matrix_table_begin' not found in first 100 lines")
    print(f"Total lines examined: {lines_read}")

# 2. Try extracting gene expression data from the matrix file again with better diagnostics
try:
    print("\nAttempting to extract gene data from matrix file...")
    gene_data = get_genetic_data(matrix_file)
    if gene_data.empty:
        print("Extracted gene expression data is empty")
        is_gene_available = False
    else:
        print(f"Successfully extracted gene data with {len(gene_data.index)} rows")
        print("First 20 gene IDs:")
        print(gene_data.index[:20])
        is_gene_available = True
except Exception as e:
    print(f"Error extracting gene data: {str(e)}")
    print("This dataset appears to have an empty or malformed gene expression matrix")
    is_gene_available = False

print(f"\nGene expression data available: {is_gene_available}")

# If data extraction failed, try an alternative approach using pandas directly
if not is_gene_available:
    print("\nTrying alternative approach to read gene expression data...")
    try:
        with gzip.open(matrix_file, 'rt') as file:
            # Skip lines until we find the marker
            for line in file:
                if '!series_matrix_table_begin' in line:
                    break
            
            # Try to read the data directly with pandas
            gene_data = pd.read_csv(file, sep='\t', index_col=0)
            
            if not gene_data.empty:
                print(f"Successfully extracted gene data with alternative method: {gene_data.shape}")
                print("First 20 gene IDs:")
                print(gene_data.index[:20])
                is_gene_available = True
            else:
                print("Alternative extraction method also produced empty data")
    except Exception as e:
        print(f"Alternative extraction failed: {str(e)}")


Examining matrix file structure...
Line 0: !Series_title	"Analysis of monocyte subsets from pretreatment RCC patients compared to healthy donors."
Line 1: !Series_geo_accession	"GSE106757"
Line 2: !Series_status	"Public on Nov 15 2019"
Line 3: !Series_submission_date	"Nov 09 2017"
Line 4: !Series_last_update_date	"Nov 15 2019"
Line 5: !Series_summary	"Peripheral blood monocytes are the starting material utilized in conventional dendritic cell (DC) vaccination for the treatment of a broad range of malignancies. While the use of cytokines and growth factors to polarize monocyte-derived DC to distinct phenotypes is well-established, little is known about the contributions of distinct human monocyte subsets to monocyte-derived DC function and patient responses to vaccination. To investigate the status of monocyte subsets in cancer patients and following culture into DC, we isolated classical (C-Mo), intermediate (I-Mo), and non-classical (NC-Mo) from the peripheral blood of renal cell carc

### Step 4: Gene Identifier Review

In [5]:
# Examining the gene identifiers from the gene data
# From the output, I can see gene identifiers like 'A1BG', 'A2M', 'AACS' which are
# standard human gene symbols. These include:
# - A1BG (Alpha-1-B Glycoprotein)
# - A2M (Alpha-2-Macroglobulin)
# - AACS (Acetoacetyl-CoA Synthetase)
# 
# These are official HGNC gene symbols, not probe IDs or other identifiers that would
# require mapping to gene symbols.

requires_gene_mapping = False


### Step 5: Data Normalization and Linking

In [6]:
# 1. Normalize gene symbols in the gene expression data
print("\nNormalizing gene symbols...")
try:
    # Check if gene_data is empty before normalization
    if gene_data.empty:
        print("Gene data is empty. Skipping normalization.")
        normalized_gene_data = gene_data
    else:
        normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
        print(f"After normalization: {len(normalized_gene_data.index)} unique gene symbols")
    
    # Save the normalized gene data (even if empty, to maintain consistent workflow)
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    normalized_gene_data.to_csv(out_gene_data_file)
    print(f"Normalized gene expression data saved to {out_gene_data_file}")
except Exception as e:
    print(f"Error normalizing gene symbols: {e}")
    normalized_gene_data = gene_data  # Use original data if normalization fails

# 2. Re-extract clinical data from matrix file to get proper sample IDs
print("\nRe-extracting clinical data with proper sample IDs...")
try:
    background_info, clinical_data = get_background_and_clinical_data(matrix_file)
    print(f"Re-extracted clinical data with shape: {clinical_data.shape}")
    
    # Find the row with disease state information
    disease_state_row = None
    sample_chars_dict = get_unique_values_by_row(clinical_data)
    
    for row_idx, values in sample_chars_dict.items():
        values_str = str(values)
        if 'disease state' in values_str.lower():
            disease_state_row = row_idx
            print(f"Found disease state information in row {row_idx}: {values}")
            break
    
    if disease_state_row is not None:
        # Extract clinical features with proper disease state row
        selected_clinical = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=disease_state_row,
            convert_trait=convert_trait
        )
        
        print(f"Selected clinical data with shape: {selected_clinical.shape}")
        
        # Save the updated clinical data
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        selected_clinical.to_csv(out_clinical_data_file)
        print(f"Updated clinical data saved to {out_clinical_data_file}")
        
        is_trait_available = True
    else:
        print("Could not find disease state information in the clinical data.")
        selected_clinical = pd.DataFrame()
        is_trait_available = False
        
except Exception as e:
    print(f"Error re-extracting clinical data: {e}")
    selected_clinical = pd.DataFrame()
    is_trait_available = False

# 3. Link clinical and genetic data if available
print("\nLinking clinical and genetic data...")
try:
    if is_trait_available and not normalized_gene_data.empty:
        # Use geo_link_clinical_genetic_data function to properly link the data
        linked_data = geo_link_clinical_genetic_data(selected_clinical, normalized_gene_data)
        print(f"Linked data shape: {linked_data.shape}")
        
        # Check if we have the trait column in the linked data
        if trait in linked_data.columns:
            trait_count = linked_data[trait].count()
            print(f"Number of samples with trait values: {trait_count}")
            
            if trait_count > 0:
                # 4. Handle missing values systematically
                print("\nHandling missing values...")
                linked_data = handle_missing_values(linked_data, trait)
                print(f"After handling missing values, data shape: {linked_data.shape}")
                
                # Check if we still have samples after missing value handling
                if linked_data.shape[0] > 0:
                    # 5. Determine whether the trait and demographic features are biased
                    print("\nChecking for bias in features...")
                    is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
                else:
                    print("Error: All samples were removed during missing value handling.")
                    is_biased = True
            else:
                print("No samples have valid trait values. Dataset cannot be used.")
                is_biased = True
        else:
            print(f"Trait column '{trait}' not found in linked data.")
            is_biased = True
    else:
        print("Cannot link data: clinical or genetic data is missing.")
        if not is_trait_available:
            print(f"Reason: Trait column '{trait}' is not available in clinical data.")
        if normalized_gene_data.empty:
            print("Reason: Gene expression data is empty.")
        linked_data = pd.DataFrame()
        is_biased = True
        
except Exception as e:
    print(f"Error in linking clinical and genetic data: {e}")
    import traceback
    traceback.print_exc()
    linked_data = pd.DataFrame()
    is_biased = True

# 6. Final quality validation
print("\nPerforming final validation...")
note = ""
if 'linked_data' in locals() and linked_data.empty:
    if not normalized_gene_data.empty and not selected_clinical.empty:
        note = "Dataset failed in linking phase. Clinical data structure could not be properly matched with gene expression data."
    elif normalized_gene_data.empty:
        note = "Dataset failed in gene mapping phase. Could not properly map probe IDs to gene symbols."
    elif selected_clinical.empty:
        note = "Dataset failed in clinical data extraction. No valid trait information available."
elif 'is_biased' in locals() and is_biased:
    note = "Dataset passed initial processing but contains severely biased trait distribution."

is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=not normalized_gene_data.empty,
    is_trait_available=is_trait_available,
    is_biased=is_biased if 'is_biased' in locals() else True,
    df=linked_data if 'linked_data' in locals() and not linked_data.empty else pd.DataFrame(),
    note=note
)

# 7. Save linked data if usable
if is_usable and 'linked_data' in locals() and not linked_data.empty:
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    
    # Save linked data
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print(f"Dataset not usable for {trait} association studies. Data not saved.")


Normalizing gene symbols...


After normalization: 20741 unique gene symbols


Normalized gene expression data saved to ../../output/preprocess/Kidney_Clear_Cell_Carcinoma/gene_data/GSE106757.csv

Re-extracting clinical data with proper sample IDs...
Re-extracted clinical data with shape: (2, 37)
Found disease state information in row 0: ['disease state: healthy', 'disease state: renal cell carcinoma']
Selected clinical data with shape: (1, 36)
Updated clinical data saved to ../../output/preprocess/Kidney_Clear_Cell_Carcinoma/clinical_data/GSE106757.csv

Linking clinical and genetic data...
Linked data shape: (36, 20742)
Number of samples with trait values: 36

Handling missing values...


After handling missing values, data shape: (36, 20742)

Checking for bias in features...
For the feature 'Kidney_Clear_Cell_Carcinoma', the least common label is '0.0' with 4 occurrences. This represents 11.11% of the dataset.
The distribution of the feature 'Kidney_Clear_Cell_Carcinoma' in this dataset is fine.


Performing final validation...


Linked data saved to ../../output/preprocess/Kidney_Clear_Cell_Carcinoma/GSE106757.csv
